In [4]:
import os

output_dir = "data_input/basemap_elements"
os.makedirs(output_dir, exist_ok=True)


## List of selected interestings layers to use for the maps

In [8]:
natural_earth_layers = [
    # Catégorie Cultural
    ("cultural", "admin_0_countries"),
    ("cultural", "admin_0_boundary_lines_land"),
    ("cultural", "admin_0_countries_lakes"),
    ("cultural", "admin_0_sovereignty"),
    ("cultural", "admin_1_states_provinces"),
    ("cultural", "admin_1_states_provinces_lakes"),
    ("cultural", "admin_1_states_provinces_lines"),
    ("cultural", "populated_places"),
    ("cultural", "urban_areas"),
    ("cultural", "roads"),
    ("cultural", "railroads"),
    # Catégorie Physical
    ("physical", "land"),
    ("physical", "ocean"),
    ("physical", "minor_islands"),
    ("physical", "reefs"),
    ("physical", "lakes"),
    ("physical", "rivers_lake_centerlines"),
    ("physical", "glaciated_areas"),
    ("physical", "antarctic_ice_shelves_polys"),
    ("physical", "antarctic_ice_shelves_lines"),
    ("physical", "coastline"),
    ("physical", "geographic_lines"),
    ("physical", "graticules_1"),
    ("physical", "graticules_5"),
    ("physical", "graticules_10"),
]
# Résolutions : "10m", "50m", "110m"


# Function

In [11]:
import os
import requests
import shutil
from zipfile import ZipFile
from pathlib import Path
import geopandas as gpd

def download_and_convert_natural_earth(layers, resolution="10m", output_dir="data_input/basemap_elements"):
    """
    Télécharge, charge, exporte en GPKG, et nettoie pour une liste de couches Natural Earth.
    On ne garde que le .gpkg final.
    """
    os.makedirs(output_dir, exist_ok=True)
    base_url_tpl = "https://naturalearth.s3.amazonaws.com/{res}_{cat}/ne_{res}_{name}.zip"

    for category, name in layers:
        url = base_url_tpl.format(res=resolution, cat=category, name=name)
        zip_path = Path(output_dir) / f"ne_{resolution}_{name}.zip"
        extract_path = Path(output_dir) / f"ne_{resolution}_{name}"
        gpkg_path = Path(output_dir) / f"ne_{resolution}_{name}.gpkg"

        print(f"\n🔻 Traitement: {name} ({category}, {resolution})")
        
        # 1. Télécharger si pas déjà là
        if not zip_path.exists():
            r = requests.get(url)
            if r.status_code != 200:
                print(f"  ❌ Erreur: {r.status_code} pour {url}")
                continue
            with open(zip_path, "wb") as f:
                f.write(r.content)
            print("  ✅ ZIP téléchargé.")
        else:
            print("  ⚡ ZIP déjà présent.")

        # 2. Dézipper
        if not extract_path.exists():
            with ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall(extract_path)
            print("  ✅ Dézippé.")
        else:
            print("  ⚡ Dossier déjà extrait.")

        # 3. Trouver le .shp principal
        shp_files = list(extract_path.glob("*.shp"))
        if not shp_files:
            print("  ❌ Pas de fichier .shp trouvé !")
            continue
        shp_path = shp_files[0]

        # 4. Charger dans GeoPandas
        try:
            gdf = gpd.read_file(shp_path)
            print("  ✅ Chargé avec GeoPandas.")
        except Exception as e:
            print(f"  ❌ Erreur lecture GeoPandas: {e}")
            continue

        # 5. Exporter en GPKG
        gdf.to_file(gpkg_path, driver="GPKG")
        print(f"  ✅ Exporté en GPKG : {gpkg_path}")

        # 6. Nettoyer : supprimer le ZIP, le dossier extrait
        try:
            os.remove(zip_path)
            print("  🗑️ ZIP supprimé.")
        except Exception:
            pass
        try:
            shutil.rmtree(extract_path)
            print("  🗑️ Dossier extrait supprimé.")
        except Exception:
            pass

    print("\n✨ Terminé ! Tous les fichiers .gpkg sont dans :", output_dir)


# Usage of function

In [ ]:
my_layers = [
    ("cultural", "admin_0_countries"),
    ("physical", "ocean"),
    ("physical", "rivers_lake_centerlines"),
    ("cultural", "populated_places"),
    ("cultural", "roads"),
    ("physical", "coastline")
]

download_and_convert_natural_earth(my_layers, resolution="10m", output_dir="data_input/basemap_elements")



🔻 Traitement: admin_0_countries (cultural, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_admin_0_countries.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: ocean (physical, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_ocean.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: rivers_lake_centerlines (physical, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_rivers_lake_centerlines.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: populated_places (cultural, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_populated_places.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: roads (cultural, 10m)
  ✅ ZIP téléchargé.
 